In [1]:
#!/usr/bin/env RScript
###################################################################
# Description: modify Yoshi & Parker 2020 paper code for Cicero 
# CCAN analysis; Then simplified the make_cicero_cds steps to 
# avoid recalculating UMAP
# The RenamePeaks step is not performed at stage to save time

# Author: Dian Li
# Last modified: 2023-June-27: 
###################################################################

In [2]:
cluster_list = c("Cortex", "Medulla", "Papilla", "Renal Artery", "Ureter")

In [3]:
library(Seurat)
library(Signac)
library(cicero)
library(SeuratWrappers)
library(dplyr)
library(here)
library(openxlsx)
library(SeuratObject)

library(monocle3)

Attaching SeuratObject

Attaching sp


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange


Loading required package: monocle3

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, ro

### create processed_data_path_new

In [4]:
processed_data_path_new = "../../processed_data/Cicero/324701_cells_by_renal_region_new"
dir.create(processed_data_path_new, recursive = T, showWarnings = F)

### create plots_path

In [5]:
plots_path = "../../plots/Cicero/324701_cells_by_renal_region_new"
dir.create(plots_path, recursive = T, showWarnings = F)

### load multiome object

In [6]:
Sys.time()
load("../../processed_data/wnn/20221221_324701_cells_wnn.RData")
Sys.time()

[1] "2023-06-28 16:27:29 CDT"

[1] "2023-06-28 16:31:46 CDT"

In [7]:
novaseq.wnn

An object of class Seurat 
237522 features across 324701 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: RNA
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP

In [8]:
DefaultAssay(novaseq.wnn) <- "peaks"

In [9]:
table(novaseq.wnn$renal_region_new)


      Cortex      Medulla      Papilla Renal Artery       Ureter 
      178521        76815        58891         3524         6950 

In [10]:
Idents(novaseq.wnn) <- "renal_region_new"
head(Idents(novaseq.wnn))

R1.034,R2.082,R3.082,P1.76,B3 R1.047,R2.073,R3.003,P1.65,B2 
                      Medulla                        Cortex 
R1.025,R2.080,R3.017,P1.66,B2 R1.056,R2.038,R3.008,P1.48,B2 
                       Cortex                       Medulla 
R1.089,R2.082,R3.024,P1.14,B1 R1.031,R2.072,R3.075,P1.78,B3 
                       Cortex                       Medulla 
Levels: Cortex Medulla Papilla Renal Artery Ureter

In [11]:
novaseq.BK = novaseq.wnn

### calculate ccan for each renal region

In [12]:
run_cicero_wrapper <- function(novaseq, clusterID, processed_data_path_new, plots_path){
   print(Sys.time())
  print("step 1.1. create a subset based on clusterID")
  
  # novaseq <- subset(novaseq, ident = clusterID) # create a subset
    if (is.null(clusterID)){
        novaseq = novaseq
        clusterID = "all_regions"
    } else{
        novaseq <- subset(novaseq, renal_region_new %in% clusterID)
    }
    
  print(paste0("Subsetting seurat object for: ",clusterID))
  print(novaseq)
    
  ###################################################################
  # convert to cicero naming convention
  print(Sys.time())
  print("step 2. convert to cicero naming convention")
  
  # for UMAP umap issue
  novaseq[["UMAP"]] <- novaseq[["umap.peaks"]]
  novaseq[["umap.peaks"]] <- NULL
  
  print(novaseq)
  
  count_data <- GetAssayData(novaseq, slot = "counts")
  summ <- summary(count_data)
  summ_frame <- data.frame(peak = rownames(count_data)[summ$i],
                           cell.id = colnames(count_data)[summ$j],
                           count = summ$x)
  print(head(summ_frame))
  ###################################################################
  # prepare cicero cell_data_set
  print(Sys.time())
  print("step 3. prepare atac cell_data_set")
  
  # create cell data set object with cicero constructor
  input_cds <- make_atac_cds(summ_frame, binarize = TRUE)  
  ###################################################################
  # step 
  # set seed. (only one time)
  print(Sys.time())
  print("step 4. set.seed(2017)")
  
  set.seed(2017)
  ###################################################################
  # step 
  # set seed. (only one time)
  print(Sys.time())
  print("step 5. make cicero cds")
  
  input_cds <- detect_genes(input_cds)
  input_cds <- estimate_size_factors(input_cds)
  
  # use novaseq original UMAP to save time and keep consistency
  input_cds <- preprocess_cds(input_cds, method="LSI")
  # input_cds <- reduce_dimension(input_cds, reduction_method="UMAP", preprocess_method="LSI")
  # 
  # umap_coords <- reducedDims(input_cds)$UMAP
  
  umap_coords = Embeddings(novaseq, reduction = "UMAP")
  cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates=umap_coords)
  
  cicero_cds
  ###################################################################
  # step 
  # prepare contigs variable
  print(Sys.time())
  print("step 6. prepare contigs variable")
  
  genome <- seqlengths(novaseq)
  
  contigs <- data.frame("V1" = names(genome), "length" = genome)  
  # select chromosomes to run cicero on
  ###################################################################
  # for testing purposes, only test chr1, chrX and chrY for now
  levels <- paste0("chr",c(seq(1,22),"X","Y"))
  # levels <- paste0("chr",c(seq(1,1),"X","Y"))
  ###################################################################
  contigs <- subset(contigs, V1 %in% levels)
  
  chrom = NULL
  # if a specific chromosome is specified, subset the contigs and only run that chromosome
  if(!is.null(chrom)) {
    contigs <- subset(contigs, V1 %in% chrom)
  }
  contigs
  ###################################################################
  # step 
  # prepare contigs variable
  print(Sys.time())
  print("step 6.1. remove varibales not being used and clean memory")
  
  rm(list = c("summ_frame", "input_cds"))
  gc()
  ###################################################################
  # step 
  # run_cicero
  print(Sys.time())
  print("step 7. run_cicero")
  
  # can the contig region be limited to 1Mb up- and downstream of gene of interest to increase speed?
  # create a subset by input chromosome
  conns <- run_cicero(cicero_cds, contigs, sample_num = 100) 
  
  save(list=c("conns"), file = file.path(processed_data_path_new, paste0("conns_ccan_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), ".RData")), compress = T)
  
  ###################################################################
  # step 
  # generate_ccans
  print(Sys.time())
  print("step 8. generate_ccans")
  
  # CCAN_assigns <- generate_ccans(conns)
  CCAN_assigns <- generate_ccans(conns, coaccess_cutoff_override = 0.2)
  
  
  
  ###################################################################
  # step 
  # save conns and ccan object
  print(Sys.time())
  print("step 9. save conns and ccan object")

  save(list=c("conns", "CCAN_assigns"), file = file.path(processed_data_path_new, paste0("conns_ccan_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), ".RData")), compress = T)
  
  ###################################################################
  # modify ccan output
  # create a column that identifies which connections belong to a CCAN
  # to save storage, won't calculate ccan for now
  if (FALSE){
    ccan1 <- left_join(conns, CCAN_assigns, by=c("Peak1" = "Peak"), all.x=TRUE)
    colnames(ccan1)[4] <- "CCAN1"
    ccan2 <- left_join(conns, CCAN_assigns, by=c("Peak2" = "Peak"), all.x=TRUE)
    colnames(ccan2)[4] <- "CCAN2"
    ccan <- cbind(ccan1, CCAN2=ccan2$CCAN2) %>%
      dplyr::mutate(CCAN = ifelse(CCAN1 == CCAN2, CCAN1, 0)) %>%
      dplyr::select(-CCAN1, -CCAN2)
    
    save(list=c("conns", "CCAN_assigns", "ccan"), file = file.path(processed_data_path_new, paste0("conns_ccan_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), ".RData")), compress = T)
  }
  ###################################################################
  # step.
  # make CoveragePlot
  links <- ConnectionsToLinks(conns = conns, ccans = CCAN_assigns)
  Links(novaseq) <- links
  ###################################################################
  p = CoveragePlot(novaseq, region = "chr1-99756821-100037935")
  
  ggsave(filename = file.path(plots_path, paste0("CoveragePlot_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), "_chr1-99756821-100037935", ".png")), plot = p, 
         width = 12, height = 10, units = "in", dpi = 300)
  ###################################################################
  # p = CoveragePlot(novaseq, region = "chrY-6996148-6997666")
  # 
  # ggsave(filename = file.path(plots_path, paste0("CoveragePlot_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), "_chrY-6996148-6997666", ".png")), plot = p, 
  #        width = 12, height = 10, units = "in", dpi = 300)
  # ###################################################################
  # p = CoveragePlot(novaseq, region = "chr1-100249518-100614716")
  # 
  # ggsave(filename = file.path(plots_path, paste0("CoveragePlot_", gsub(pattern = "\\/", "_", paste(clusterID, collapse = "_")), "_chr1-100249518-100614716", ".png")), plot = p, 
  #        width = 12, height = 10, units = "in", dpi = 300) 
    print(Sys.time())
}

In [ ]:
###################################################################
###################################################################
###################################################################
# start for loop

# for (clusterID in unique(novaseq$celltype)){
for (clusterID in cluster_list){
  # convert to cicero naming convention
  run_cicero_wrapper(novaseq = novaseq.BK, clusterID = clusterID, 
             processed_data_path_new = processed_data_path_new, 
             plots_path = plots_path)
  
}
  
###################################################################
###################################################################
###################################################################


[1] "2023-06-28 16:31:47 CDT"
[1] "step 1.1. create a subset based on clusterID"
[1] "Subsetting seurat object for: Cortex"
An object of class Seurat 
237522 features across 178521 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: RNA
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP
[1] "2023-06-28 16:33:31 CDT"
[1] "step 2. convert to cicero naming convention"


Warning message:
"Cannot add objects with duplicate keys (offending key: peaksUMAP_), setting key to 'umap_'"


An object of class Seurat 
237522 features across 178521 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: RNA
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, WNN.UMAP, UMAP
                  peak                       cell.id count
1     chr1-10367-10542 R1.047,R2.073,R3.003,P1.65,B2     1
2 chr1-7705359-7705897 R1.047,R2.073,R3.003,P1.65,B2     1
3 chr1-7727461-7728046 R1.047,R2.073,R3.003,P1.65,B2     1
4 chr1-8021056-8022096 R1.047,R2.073,R3.003,P1.65,B2     1
5 chr1-8181000-8181670 R1.047,R2.073,R3.003,P1.65,B2     1
6 chr1-8908626-8909181 R1.047,R2.073,R3.003,P1.65,B2     1
[1] "2023-06-28 16:33:57 CDT"
[1] "step 3. prepare atac cell_data_set"
[1] "2023-06-28 16:41:25 CDT"
[1] "step 4. set.seed(2017)"
[1] "2023-06-28 16:41:25 CDT"
[1] "step 5. make cicero cds"


Overlap QC metrics:
Cells per bin: 50
Maximum shared cells bin-bin: 44
Mean shared cells bin-bin: 0.0135225265040003
Median shared cells bin-bin: 0



### run Cicero for all regions combined

In [ ]:
run_cicero_wrapper(novaseq = novaseq.BK, clusterID = NULL, 
             processed_data_path_new = processed_data_path_new, 
             plots_path = plots_path)

In [ ]:
Sys.time()